## Exercise 7.2

In this chapter, I showed how we can express the DFT and inverse DFT
as matrix multiplications.  These operations take time proportional to
$N^2$, where $N$ is the length of the wave array.  That is fast enough
for many applications, but there is a faster
algorithm, the Fast Fourier Transform (FFT), which takes time
proportional to $N \log N$.

The key to the FFT is the Danielson-Lanczos lemma:

$DFT(y)[n] = DFT(e)[n] + \exp(-2 \pi i n / N) DFT(o)[n]$

Where $ DFT(y)[n]$ is the $n$th element of the DFT of $y$; $e$ is the even elements of $y$, and $o$ is the odd elements of $y$.

This lemma suggests a recursive algorithm for the DFT:

1. Given a wave array, $y$, split it into its even elements, $e$, and its odd elements, $o$.

2. Compute the DFT of $e$ and $o$ by making recursive calls.

3. Compute $DFT(y)$ for each value of $n$ using the Danielson-Lanczos lemma.

For the base case of this recursion, you could wait until the length
of $y$ is 1.  In that case, $DFT(y) = y$.  Or if the length of $y$
is sufficiently small, you could compute its DFT by matrix multiplication,
possibly using a precomputed matrix.

Hint: I suggest you implement this algorithm incrementally by starting
with a version that is not truly recursive.  In Step 2, instead of
making a recursive call, use `dft` or `np.fft.fft`.  Get Step 3 working,
and confirm that the results are consistent with the other
implementations.  Then add a base case and confirm that it works.
Finally, replace Step 2 with recursive calls.

One more hint: Remember that the DFT is periodic; you might find `np.tile` useful.

You can read more about the FFT at https://en.wikipedia.org/wiki/Fast_Fourier_transform.

In [1]:
import numpy as np
PI2 = 2 * np.pi

Sample signal:

In [10]:
ys = [0.6, 0.25, 0.1, 0.05, 0.3, 0.2,-0.5,0.3]

Function to compare results of FT:

In [3]:
def compare(arr1, arr2):
    result = np.sum(np.abs(arr1-arr2))
    return format(result, '.1g')

Step 2: We use np.fft.fft instead of making a recursive call to implement the algorithm:

In [13]:
def fake_fft(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

We compare results of this implementation with np.fft.fft:

In [14]:
compare(fake_fft(ys), np.fft.fft(ys))

'3e-16'

The error is significantly small, so we can confirm that the function works.

Step 3: Now we try replacing Step 2 with recursive calls and adding base case to the function:

In [6]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys
    
    He = fft(ys[::2])
    Ho = fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

Compare the results of implementations:

In [11]:
compare(fft(ys), np.fft.fft(ys))

'1e-15'

They are the same.